In [1]:
%pip install newsapi-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from newsapi import NewsApiClient
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import openai
import torch

c:\Users\David\anaconda3\envs\TEMG4950L\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# API configuration
openai.api_type = "azure"
openai.api_base = "https://gpt-722.openai.azure.com/"
openai.api_key = "bbdc656728034b52870b2618e2cb36cb"
openai.api_version = "2023-09-01-preview"
MODEL = "GPT-4-for-EDU"
news_api_key = "f8f1f4e2c57a453fa84c593c3aff7494"
newsapi = NewsApiClient(api_key=news_api_key)

In [4]:
# Get all useful sources
sources = newsapi.get_sources(language='en')
sources = sources['sources']
sources = pd.DataFrame(sources)
sources

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
1,abc-news-au,ABC News (AU),"Australia's most trusted source of local, nati...",http://www.abc.net.au/news,general,en,au
2,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,en,us
3,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,en,us
4,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
...,...,...,...,...,...,...,...
76,the-washington-times,The Washington Times,The Washington Times delivers breaking news an...,https://www.washingtontimes.com/,general,en,us
77,time,Time,Breaking news and analysis from TIME.com. Poli...,http://time.com,general,en,us
78,usa-today,USA Today,"Get the latest national, international, and po...",http://www.usatoday.com/news,general,en,us
79,vice-news,Vice News,"Vice News is Vice Media, Inc.'s current affair...",https://news.vice.com,general,en,us


In [5]:
# Concatenate the sources into a string for api call
listOfSources = ""
for i in range(len(sources)):
    listOfSources += f'{sources["id"][i]}, '
# results = newsapi.get_everything(q='+ETF AND (launch OR list OR delist)',language='en',sources=listOfSources,sort_by='publishedAt',from_param='2023-10-05', to='2023-11-04')
# results = results['articles']
# results

In [6]:
# Function to extract news articles from the results into a txt file for further processing
# def extract_content(url):
#     driver = webdriver.Chrome()
#     driver.get(url)
#     title = driver.title
#     content = driver.find_elements('xpath', '//script[]')
#     content_list = []
#     for i in content:
#         content_list.append(i.text)
#     content_list = " ".join(content_list)
#     driver.close()
#     return [title, content_list]

In [7]:
# Get today's date and the date 31 days ago
from datetime import datetime, timedelta
endtime = datetime.today().strftime('%Y-%m-%d')
starttime = (datetime.today() - timedelta(days=31)).strftime('%Y-%m-%d')
print(starttime, endtime)

2023-10-08 2023-11-08


In [22]:
# finbert query function
def finbert_query(text):
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

    result = nlp(text)
    return result[0].get('label')

# GPT summary function
def gpt_summary(title, description, content):
    response = openai.ChatCompletion.create(
        engine=MODEL,
        messages=[
            {"role": "system", "content": "You are a financial analyst. You are reading a news article about a recent ETF launch. You need to summarize the article and write a short report about it for investment specialists and executives in JPMC's ETF sales team."},
            {"role": "user", "content": f"Title: {title}"},
            {"role": "user", "content": f"Description: {description}"},
            {"role": "user", "content": f"Description: {content}"}
        ],
        temperature=0.2,
    )
    return response['choices'][0]['message']['content']

In [42]:
launching_result = requests.get(f'https://newsapi.org/v2/everything/?q=ETF AND (launch OR list OR delist)&sources={listOfSources}&searchIn=title,description&language=en&sortBy=publishedAt&from={starttime}&to={endtime}&apiKey={news_api_key}')
launching_result = launching_result.json()
export_result = dict(sorted(launching_result.items()))
export_result = json.dumps(launching_result, indent=4)

{'status': 'ok', 'totalResults': 2, 'articles': [{'source': {'id': 'the-times-of-india', 'name': 'The Times of India'}, 'author': 'Bloomberg', 'title': 'Bitcoin’s 10% jump to $30,000 hints at trader ‘playbook’ for ETF launches', 'description': 'An erroneous report that BlackRock Inc. had won approval to launch a spot ETF rapidly sent the largest digital asset to $30,002 on Monday, the highest price since March. The move cooled after the world’s biggest money manager said its application remains unde…', 'url': 'https://economictimes.indiatimes.com/markets/cryptocurrency/bitcoins-10-jump-to-30000-hints-at-trader-playbook-for-etf-launches/articleshow/104490656.cms', 'urlToImage': 'https://img.etimg.com/thumb/msid-104490656,width-1200,height-630,imgsize-2381235,overlay-etmarkets/photo.jpg', 'publishedAt': '2023-10-17T07:44:55Z', 'content': 'A brief 10% surge in Bitcoin gave traders a glimpse into the possible impact of a looming US Securities &amp; Exchange Commission decision on whether t

In [21]:
# API call for ETF launch news, newsapi library does not support searching only title
launching_result = requests.get(f'https://newsapi.org/v2/everything/?q=ETF AND (launch OR list OR delist)&sources={listOfSources}&searchIn=title,description&language=en&sortBy=publishedAt&from={starttime}&to={endtime}&apiKey={news_api_key}')
launching_result = launching_result.json()
export_result = dict(sorted(launching_result.items()))
export_result = json.dumps(launching_result, indent=4)

# Export the results to a json file
with open('launching_result.json', 'w') as f:
    f.write(export_result)

num_results = launching_result['totalResults']
launching_result = launching_result['articles']
launching_result = pd.DataFrame(launching_result)
launching_result = launching_result.drop_duplicates(subset=['title'])
launching_result = launching_result.reset_index(drop=True)
launching_result = launching_result.drop(columns=['source', 'author', 'url', 'urlToImage', 'publishedAt'])
analysis_list = []
analysis_result = pd.DataFrame(columns=['title', 'summary', 'sentiment'], index=range(num_results))
for i in range(num_results):
    temp = launching_result.iloc[i]
    analysis_list.append(temp['title']+'\n' + temp['description']+ '\n'+ temp['content'])
    analysis_result.iloc[i] = [temp['title'], finbert_query(analysis_list[i]), gpt_summary(temp['title'],temp['description'],temp['content'])]
analysis_result


[{'label': 'Negative', 'score': 0.9785405397415161}]
Summary:

The recent news of BlackRock Inc. potentially receiving approval to launch a spot ETF resulted in a 10% surge in Bitcoin's price, reaching $30,002 - the highest since March. However, this was based on an erroneous report, as BlackRock confirmed that its application is still under review. This incident provided an insight into the potential impact of the upcoming decision by the U.S. Securities & Exchange Commission (SEC) on the approval of exchange-traded funds (ETFs) that directly invest in Bitcoin. 

This event suggests that the launch of such ETFs could significantly influence the price of Bitcoin and other digital assets. It also highlights the importance of accurate information in the highly sensitive cryptocurrency market. The SEC's decision on Bitcoin ETFs is eagerly anticipated and could potentially open up new investment opportunities in the digital asset space.
[{'label': 'Positive', 'score': 0.9770365357398987}]


,title,summary,sentiment
0,"Bitcoin’s 10% jump to $30,000 hints at trader ...",Negative,Summary:\n\nThe recent news of BlackRock Inc. ...
1,"Bitcoin jumps 10%, then slides after BlackRock...",Positive,Summary: \n\nThe article discusses the recent ...


In [47]:
# Export the results to a json file
analyzed_result = analysis_result.to_dict(orient='records')
with open('analyzed_result.json', 'w') as f:
    for i in range(len(analyzed_result)):
        single_result = dict(analyzed_result[i].items())
        single_result = json.dumps(single_result, indent=4)
        f.write(single_result)
        f.write('\n')
    f.close()